Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [X] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


Import libraries section

In [4]:
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [5]:
### Split train into train & val

train, val = train_test_split(train,train_size=0.8, test_size=0.2,
                             stratify=train['status_group'],random_state=42)

train.shape, val.shape, test.shape


((47520, 41), (11880, 41), (14358, 40))

In [6]:
### Process and some data clean

def wrangle(X):
    """"Wrangle train, validate and test sets in the same way """
    
    ###Present settingWithCopyWarning
    X=X.copy()
    
    ### Gattin rid of latitude values really near to zero.
    X['latitude'] = X['latitude'].replace(-2e-08,0)
    
    ### Changing the zeros of some columns to NaN's
    ### Create a 'missing indicator' column because the fact
    ### the value is missing may be an indicator.
    cols_with_zeros = ['longitude','latitude','construction_year']
    
    for col in cols_with_zeros:
        X[col] = X[col].replace(0,np.nan)
        X[col + '_MISSING'] = X[col].isnull()
        
    
    ### Drop duplicated columns
    
    duplicates = ['quantity_group', 'payment_type', 'extraction_type']
    X = X.drop(columns=duplicates)
    
    ### Drop recorder_by (never varies), and id (always varies, random)
    unusable_variance = ['recorded_by','id']
    X = X.drop(columns=unusable_variance)
    
    ###Drop some columns with possible non relevance in the data.
    poss_nonrel = ['funder', 'public_meeting', 'installer', 'permit','wpt_name',
                 'region','lga','scheme_name','district_code','source_type',
                 'num_private','ward']
    X = X.drop(columns=poss_nonrel)
    
    ### Convert data_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    
    ### Extract components from data_recorded, then drop the original column.
    X['year_recorded']= X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    ###Engenieer features: how many years from construction_year to date recorded
    
    X['years'] = X['year_recorded'] - X['construction_year']
    
    X['years_MISSING'] = X['years'].isnull()
    
    ##return the wrangled dataframe
    
    return X
    

In [7]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [8]:
### Consider taking some outliers out.

In [9]:
### our taget is status_group

target = 'status_group'
features = train.columns.drop(target)

features

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'basin',
       'subvillage', 'region_code', 'population', 'scheme_management',
       'construction_year', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'water_quality',
       'quality_group', 'quantity', 'source', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'longitude_MISSING',
       'latitude_MISSING', 'construction_year_MISSING', 'year_recorded',
       'month_recorded', 'day_recorded', 'years', 'years_MISSING'],
      dtype='object')

In [10]:
### Arrange data into X features matrix and y target vector.

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]



In [11]:
### Randomized Search CV

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=0)
)

param_distributions = {
    
    'simpleimputer__strategy': ['median'],
    'randomforestclassifier__n_estimators': randint(50,500),
    'randomforestclassifier__max_features': uniform(0,1),
    'randomforestclassifier__criterion': ['gini','entropy']
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    return_train_score=True,
    n_jobs=-1
    
)

In [12]:
search.fit(X_train,y_train);

In [13]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', search.best_score_)

Best hyperparameters {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_features': 0.31696407048073494, 'randomforestclassifier__n_estimators': 147, 'simpleimputer__strategy': 'median'}
Cross-validation MAE 0.7983164983164983


# Draft

In [14]:
train.shape

(47520, 31)

In [15]:
train.describe()

,amount_tsh,gps_height,longitude,latitude,region_code,population,construction_year,year_recorded,month_recorded,day_recorded,years
count,47520.000000,47520.000000,46078.000000,46078.000000,47520.000000,47520.000000,31003.000000,47520.000000,47520.000000,47520.000000,31003.000000
mean,321.925261,669.567656,35.149033,-5.884512,15.258291,179.805513,1996.825469,2011.921907,4.366604,15.625926,15.343289
std,3197.240487,693.005745,2.604241,2.805599,17.530228,463.081564,12.499247,0.959690,3.025041,8.685898,12.524152
min,0.000000,-63.000000,29.607122,-11.649440,1.000000,0.000000,1960.000000,2002.000000,1.000000,1.000000,-7.000000
25%,0.000000,0.000000,33.284679,-8.633876,5.000000,0.000000,1988.000000,2011.000000,2.000000,8.000000,4.000000
50%,0.000000,372.500000,35.008578,-5.170151,12.000000,25.000000,2000.000000,2012.000000,3.000000,16.000000,13.000000
75%,25.000000,1320.000000,37.223501,-3.375068,17.000000,215.000000,2008.000000,2013.000000,7.000000,23.000000,25.000000
max,350000.000000,2770.000000,40.344301,-0.998464,99.000000,15300.000000,2013.000000,2013.000000,12.000000,31.000000,53.000000


In [16]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
years_MISSING,47520,2,False,31003
latitude_MISSING,47520,2,False,46078
longitude_MISSING,47520,2,False,46078
construction_year_MISSING,47520,2,False,31003
status_group,47520,3,functional,25807
source_class,47520,3,groundwater,36638
management_group,47520,5,user-group,42027
quantity,47520,5,enough,26567
waterpoint_type_group,47520,6,communal standpipe,27642
quality_group,47520,6,good,40598


In [17]:
train.nunique()

amount_tsh                      94
gps_height                    2401
longitude                    46028
latitude                     46026
basin                            9
subvillage                   17231
region_code                     27
population                     986
scheme_management               12
construction_year               54
extraction_type_group           13
extraction_type_class            7
management                      12
management_group                 5
payment                          7
water_quality                    8
quality_group                    6
quantity                         5
source                          10
source_class                     3
waterpoint_type                  7
waterpoint_type_group            6
status_group                     3
longitude_MISSING                2
latitude_MISSING                 2
construction_year_MISSING        2
year_recorded                    5
month_recorded                  12
day_recorded        